In [1]:
import pandas as pd
from math import sqrt
import datetime
import numpy as np
import re

In [2]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/animesDF.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores.csv')

In [7]:
print(usersDF.shape, animesDF.shape, scoresDF.shape)

(108711, 17) (2084, 33) (3448588, 4)


In [3]:
anime_ids = np.sort(animesDF.anime_id.unique())
len(anime_ids)

2084

In [4]:
top_10000_users = list(usersDF.sort_values(by=['user_completed'], ascending=False).head(10000)['username'])

In [53]:
#scoresDF = scoresDF.sample(n=10000, random_state=15)
#scoresDF.to_csv('../../../Downloads/myanimelist/scores_sample.csv', index=False)

In [5]:
scoresDF = scoresDF[scoresDF['username'].isin(top_10000_users)]

In [6]:
scoresDF = scoresDF[scoresDF['anime_id'].isin(anime_ids)]

In [53]:
animesDF = animesDF[(animesDF['duration_min']>=20) \
                    & (animesDF['aired_from_year']>=2000) \
                    & ~(animesDF.related.str.contains('Prequel')) \
                    & (animesDF['type'] != 'Special') \
                    & ~(animesDF.genre.str.contains('Kids')) \
                    & ~(animesDF.genre.str.contains('Shoujo Ai')) \
                    & ~(animesDF.genre.str.contains('Shounen Ai')) \
                    & ~(animesDF.genre.str.contains('Yaoi')) \
                    & ~(animesDF.genre.str.contains('Yuri'))]
animesDF.to_csv('../../../Downloads/myanimelist/animesDF.csv', index=False)

In [8]:
animesDF.sort_values(by=['anime_id']).head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
1622,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0
1490,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,Tuesdays at Unknown,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0
1366,8,Beet the Vandel Buster,Beet the Vandel Buster,冒険王ビィト,"Adventure King Beet, Bouken Ou Beet",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,52,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",NaN,NaN,Toei Animation,"Adventure, Fantasy, Shounen, Supernatural","['#1: ""Emotion"" by Sunbrain', '#2: ""Wish Men"" ...","['#1: ""pureness"" by Nana Kitade (eps 1-13)', '...",23.0,2004.0
885,15,Eyeshield 21,NaN,アイシールド21,Eyeshield21,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,145,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 43, 'type': 'manga'...","TV Tokyo, Nihon Ad Systems, TV Tokyo Music, Sh...",Sentai Filmworks,Studio Gallop,"Action, Sports, Comedy, Shounen","['#1: ""Breakthrough"" by Coming Century (eps 1-...","['#1: ""Be Free"" by Rikken&#039;z (eps 1-13)', ...",23.0,2005.0
214,16,Hachimitsu to Clover,Honey and Clover,ハチミツとクローバー,"HachiKuro, Honey & Clover",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,24,Finished Airing,...,Fridays at 00:35 (JST),"{'Adaptation': [{'mal_id': 1009, 'type': 'mang...","Genco, Fuji TV, Shueisha",Viz Media,J.C.Staff,"Comedy, Drama, Josei, Romance, Slice of Life","['""Dramatic"" by Yuki']","['#1: ""Waltz"" by Suneohair (eps 1-12, 24)', '#...",23.0,2005.0


In [9]:
scoresDF.head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [10]:
anime_users = scoresDF.pivot(index='anime_id', columns='username', values='my_score')

In [11]:
'''
gets all user scores
'''
def get_user_scores(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

get_user_scores(scoresDF, 'MistButterfly').head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [12]:
# rewrite all with new dataframe, speed increase expected
def sim_pearson(anime_users, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    both_scores = anime_users.loc[[anime1_id, anime2_id],:].dropna(axis='columns').to_numpy()
    scores_anime1 = both_scores[0]
    scores_anime2 = both_scores[1]
    
    shared_viewers = len(scores_anime1)
    
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = np.sum(scores_anime1)
    sum2 = np.sum(scores_anime2)
    
    # sum up the squares
    sum1Sq = np.sum(scores_anime1**2)
    sum2Sq = np.sum(scores_anime2**2)
    
    # sum up the products
    #pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    pSum = np.dot(scores_anime1, scores_anime2)
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return round(num/den, 10)

In [13]:
sim_pearson(anime_users, 9260, 2605)

0.32120281

In [14]:
'''
builds table of similarity of the shows
a[i][j] - similarity of the shows with ids: anime_ids[i] and anime_ids[j]
'''
def get_similarity_table(scores, anime_ids):
    similarity_table = np.load('similarity_table.npy')
    n = len(anime_ids)
    counter1 = 0
    counter2 = 0
    for anime1_id in anime_ids:
        counter2 = 0
        for anime2_id in anime_ids:
            if anime1_id != anime2_id:
                try:
                    similarity = sim_pearson(scores, anime1_id, anime2_id)
                except:
                    similarity = 0
                similarity_table[counter1][counter2] = similarity
                similarity_table[counter2][counter1] = similarity
                counter2 += 1
        counter1 += 1
        print('Processed {} of {} animes :: {}'.format(counter1, n, datetime.datetime.now()))
        np.save('similarity_table', similarity_table)
    return similarity_table

In [127]:
print(datetime.datetime.now())
get_similarity_table(anime_users, anime_ids)

2020-02-03 16:26:03.679100
Processed 1884 of 2084 animes :: 2020-02-03 16:26:11.623198
Processed 1885 of 2084 animes :: 2020-02-03 16:26:19.294286
Processed 1886 of 2084 animes :: 2020-02-03 16:26:26.150058
Processed 1887 of 2084 animes :: 2020-02-03 16:26:33.887477
Processed 1888 of 2084 animes :: 2020-02-03 16:26:42.103070
Processed 1889 of 2084 animes :: 2020-02-03 16:26:54.187567
Processed 1890 of 2084 animes :: 2020-02-03 16:27:04.361663
Processed 1891 of 2084 animes :: 2020-02-03 16:27:16.630238
Processed 1892 of 2084 animes :: 2020-02-03 16:27:26.798177
Processed 1893 of 2084 animes :: 2020-02-03 16:27:36.343623
Processed 1894 of 2084 animes :: 2020-02-03 16:27:47.309862
Processed 1895 of 2084 animes :: 2020-02-03 16:27:55.797878
Processed 1896 of 2084 animes :: 2020-02-03 16:28:05.680259
Processed 1897 of 2084 animes :: 2020-02-03 16:28:15.568679
Processed 1898 of 2084 animes :: 2020-02-03 16:28:27.119809
Processed 1899 of 2084 animes :: 2020-02-03 16:28:37.995801
Processed 190

Processed 2021 of 2084 animes :: 2020-02-03 16:44:45.264503
Processed 2022 of 2084 animes :: 2020-02-03 16:44:52.525904
Processed 2023 of 2084 animes :: 2020-02-03 16:44:59.651261
Processed 2024 of 2084 animes :: 2020-02-03 16:45:07.295327
Processed 2025 of 2084 animes :: 2020-02-03 16:45:14.278505
Processed 2026 of 2084 animes :: 2020-02-03 16:45:21.484366
Processed 2027 of 2084 animes :: 2020-02-03 16:45:28.663344
Processed 2028 of 2084 animes :: 2020-02-03 16:45:35.746309
Processed 2029 of 2084 animes :: 2020-02-03 16:45:43.114525
Processed 2030 of 2084 animes :: 2020-02-03 16:45:50.212885
Processed 2031 of 2084 animes :: 2020-02-03 16:45:57.456232
Processed 2032 of 2084 animes :: 2020-02-03 16:46:04.652750
Processed 2033 of 2084 animes :: 2020-02-03 16:46:12.152190
Processed 2034 of 2084 animes :: 2020-02-03 16:46:19.453424
Processed 2035 of 2084 animes :: 2020-02-03 16:46:26.872571
Processed 2036 of 2084 animes :: 2020-02-03 16:46:34.035996
Processed 2037 of 2084 animes :: 2020-02

array([[ 0.30246524,  0.30246524,  0.24109927, ...,  0.        ,
         0.23970106,  0.42921624],
       [ 0.30246524,  0.21368076,  0.21368076, ...,  0.        ,
         0.54772256,  0.47719134],
       [ 0.24109927,  0.21368076,  0.53160617, ...,  0.        ,
         1.        , -0.21821789],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.23970106,  0.54772256,  1.        , ...,  0.        ,
         0.69266029,  0.69266029],
       [ 0.42921624,  0.47719134, -0.21821789, ...,  0.        ,
         0.69266029,  0.        ]])

In [15]:
'''
loads user's scores from MAL xml file converted to csv
'''
def load_user_list(path):
    userDF = pd.read_csv(path)
    return userDF[['series_animedb_id', 'my_score']][userDF['my_score']!=0]

In [16]:
cmlimmDF = load_user_list('../../../Downloads/animelist_cmlimm.csv')

In [89]:
barbieDF = load_user_list('../../../Downloads/animelist_barbie.csv')

In [17]:
'''
find user's shows that exist in MAL dataset, as some shows aired after dataset was created
'''
def user_shows_in_db(userDF, animesDF):
    return userDF[userDF[['series_animedb_id']].isin(animesDF.anime_id.unique()).any(1)]

In [20]:
np.array(user_shows_in_db(cmlimmDF, animesDF))

array([[22147,     9],
       [ 2251,    10],
       [35965,     7],
       [28805,     7],
       [31964,     8],
       [33929,     8],
       [35459,     8],
       [16592,     6],
       [35849,     8],
       [ 6746,     8],
       [ 9314,     7],
       [33731,     5],
       [ 2890,     7],
       [18229,     7],
       [20583,     9],
       [25303,     7],
       [35806,     7],
       [27989,    10],
       [30415,     8],
       [10357,     9],
       [ 5680,     6],
       [ 9617,     6],
       [32686,     7],
       [18679,     9],
       [32281,     8],
       [31798,     4],
       [28851,     9],
       [30831,     7],
       [34712,     9],
       [33489,     9],
       [35677,     7],
       [15051,     7],
       [32526,     2],
       [34599,     8],
       [21603,     5],
       [10161,     8],
       [21557,     7],
       [31559,     6],
       [ 6675,     5],
       [ 5671,     7],
       [10884,     7],
       [29786,     6],
       [16498,     6],
       [ 61

In [83]:
'''
finds top matches based on shows similarities
'''
def top_matches(similarity_table, user_show_scores, anime_ids):
    result = np.empty((0,2), int)
    user_shows = user_show_scores[:, 0]
    user_scores = user_show_scores[:, 1]
    rows = np.where(np.isin(anime_ids, user_shows))[0]
    columns = np.setdiff1d(np.arange(len(anime_ids)), rows)
    matches_table = similarity_table[rows][:, columns]
    for i in range(len(columns)):
        result = np.append(result, [[anime_ids[columns[i]], np.dot(matches_table[:, i], user_scores)/np.sum(matches_table[:, i])]], axis=0)
    return result[result[:,1].argsort()][::-1]

In [25]:
similarity_table = np.load('similarity_table.npy')

In [86]:
def show_names(matches, animesDF):
    for show in matches:
        print(animesDF[animesDF['anime_id']==show[0]]['title'].values[0], show[1])

In [90]:
show_names(top_matches(similarity_table, np.array(user_shows_in_db(barbieDF, animesDF)), anime_ids), animesDF)

/Users/cmlimm/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


Tensai? Dr. Hamax nan
Aurora wo Mita Kyouryuu-tachi nan
Kahei no Umi nan
"Eikou Naki Tensai-tachi" Kara no Monogatari nan
Maemilggot, Unsu Joeun Nal, Geurigo Bombom nan
Ore to Issho ni Training nan
Cocolors nan
Tensai Bakavon: Yomigaeru Flanders no Inu nan
Kouchuu Ouja Mushiking: Greatest Champion e no Michi nan
Da Hu Fa nan
Xia Lan nan
Taka no Tsume 7: Joou Heika no Jobuubu nan
Mao Yu Tao Hua Yuan nan
Zettai Yareru Greece Shinwa 20.973061347268082
Dai Yamato Zero-gou 10.45252851939693
Sinbad: Soratobu Hime to Himitsu no Shima 10.451744646984908
Mahou Shoujo? Naria☆Girls Nama de Anime wo Tsukuru Sama 10.317290437729362
Kaitou Tenshi Twin Angel: Kyun Kyun☆Tokimeki Paradise!! OVA 9.439414379753565
Firestorm 9.108455130216791
Negibouzu no Asatarou 8.639472670305343
Rilu Rilu Fairilu: Yousei no Door 8.273306134415265
Tsuri Baka Nisshi 8.25
Xue Se Cang Qiong 8.146310542008896
Kacchikenee! 8.124581690026154
Dan Doh!! 8.103479653388888
Gokujou!! Mecha Mote Iinchou 7.99317822693457
Tokyo Requi

Young Black Jack 7.314421741263461
Gun Frontier 7.314348245020356
Wizard Barristers: Benmashi Cecil 7.31308104185533
Flag 7.312320743454386
Panchira Teacher 7.3120917684116655
Dance with Devils 7.31135833701234
Gun x Sword 7.310759849783005
Suite Precure♪ 7.310728471276915
Yamato Nadeshiko Shichihenge♥ 7.3105766073911775
Vatican Kiseki Chousakan 7.310164651740459
Sensei no Ojikan: Doki Doki School Hours 7.309992109181511
Gegege no Kitarou (2018) 7.30991319841063
Kakyuusei 2: Hitomi no Naka no Shoujo-tachi 7.309734068072468
Koi☆Sento 7.309462991933669
Sousei no Aquarion 7.309338331730852
Peach Girl 7.309037462558467
Mitsudomoe 7.308956875120835
Juushin Enbu: Hero Tales 7.308930839912643
Yumeria 7.308108880337801
Yuugo: Koushounin 7.307851035092148
Captain Herlock 7.307698760255115
Yowamushi Pedal: Re:RIDE 7.307507350596541
Bakumatsu Rock 7.307053769545972
Meganebu! 7.306399923303273
Ginga e Kickoff!! 7.305730152267202
Gugure! Kokkuri-san 7.305683870083546
Hi no Tori 7.305328539375798
Sa

Saki Biyori Animation 7.2608265981435975
Divine Gate 7.26073820569675
eX-Driver: Nina & Rei Danger Zone 7.260646221671125
Tennis no Ouji-sama 7.260534532261057
Tales of Phantasia The Animation 7.260390326864832
Honoo no Haramase Oppai: Ero Appli Gakuen The Animation 7.260096072471947
Hamatora The Animation 7.260048069579981
All Out!! 7.259907171633449
Master of Epic: The Animation Age 7.25981740446277
Kiddy Grade: Ignition 7.259602112982693
Yozakura Quartet: Hana no Uta 7.259573714775925
Sekai Seifuku: Bouryaku no Zvezda 7.259318452504374
D-Frag! 7.259255595543504
Junketsu no Maria 7.259138488326432
Ueki no Housoku 7.259004299245442
Sin: Nanatsu no Taizai 7.258942564110424
Miira no Kaikata 7.2589421312806435
Natsume Yuujinchou: Itsuka Yuki no Hi ni 7.258867746890136
Gekkan Shoujo Nozaki-kun 7.258790986882936
Inari, Konkon, Koi Iroha. 7.258600974684987
Kenzen Robo Daimidaler 7.258224763116868
Fuujin Monogatari 7.258167717096105
Seiken no Blacksmith 7.2581290728164385
Kyougoku Natsuhiko:

Namiuchigiwa no Muromi-san OVA 7.232353372447388
Tayutama: Kiss on My Deity 7.232338457790445
Arai Men to YuYu The Animation 7.232323149334281
Layton Kyouju to Eien no Utahime 7.232286818573017
Amaama to Inazuma 7.232140368100772
Aoi Bungaku Series 7.23204611989253
InuYasha: Toki wo Koeru Omoi 7.231957015532667
Kage (OVA) 7.231762218446848
Starship Operators 7.231576618694705
Sumomomo Momomo: Chijou Saikyou no Yome 7.2315316613830865
Hyakko 7.23145714319057
Persona 3 the Movie 1: Spring of Birth 7.2314389478627366
Tsugumomo 7.231396103344602
Ramen Daisuki Koizumi-san 7.231275284523488
Tetsuwan Birdy Decode 7.231217053008846
Chocotto Sister 7.231009181977484
Sword Gai: The Animation 7.2308334746473015
Baka to Test to Shoukanjuu 7.230831074765357
Yoku Wakaru Gendai Mahou 7.230623767502612
Lamune 7.2304847874939755
Kyouran Kazoku Nikki 7.230329440330041
Itsuka Tenma no Kuro Usagi 7.230240018323812
Ookami Kodomo no Ame to Yuki 7.230169798984461
Outbreak Company 7.23016495518036
Hundred 7.2

InuYasha: Guren no Houraijima 7.213475346125173
Ghost in the Shell: Arise - Border:1 Ghost Pain 7.213349824268082
Suika 7.213244849972606
Kishin Houkou Demonbane 7.213168330094285
Ookiku Furikabutte 7.2131398429514375
Show By Rock!! 7.213003621063976
Prism Ark 7.213002079940178
Jinrui wa Suitai Shimashita 7.212984899966508
Bokurano 7.212891570247049
Star Driver the Movie 7.2128911844665184
Giovanni no Shima 7.212832449466508
Kara no Kyoukai 1: Fukan Fuukei 7.212698268015372
Taishou Yakyuu Musume. 7.212644123677097
Seitokai no Ichizon 7.212618962152357
Kiddy Grade 7.2125757592243955
Owari no Seraph 7.212554470057511
Ito Junji: Collection 7.2124195793009385
Ayakashi 7.212229759722
Tenjoubito to Akutobito Saigo no Tatakai 7.212099362288392
Kyou no 5 no 2 (TV) 7.211994395164406
Arcade Gamer Fubuki 7.21187499982918
Sugar Sugar Rune 7.211833088565877
Absolute Duo 7.211666612368252
Cyborg 009: The Cyborg Soldier 7.211661609170695
Macademi Wasshoi! 7.21164337025211
Sora no Method 7.21158133529

Kannagi 7.196456308938177
Demashita! Powerpuff Girls Z 7.196401767296043
Tetsuko no Tabi 7.196319300279563
Fate/Apocrypha 7.196223875863752
Hitsuji no Uta 7.196123471534814
Shakunetsu no Takkyuu Musume 7.1959319412814615
Toaru Majutsu no Index Movie: Endymion no Kiseki 7.195874372588275
Yahari Ore no Seishun Love Comedy wa Machigatteiru. OVA 7.195766058617596
Minami Kamakura Koukou Joshi Jitenshabu 7.195738838417963
Les Misérables: Shoujo Cosette 7.195701760884251
Juuni Kokuki 7.195597824372932
Inou-Battle wa Nichijou-kei no Naka de 7.195526849850191
Fullmetal Alchemist: The Sacred Star of Milos 7.19544601307164
Rail Wars! 7.1954382148286635
Saikin, Imouto no Yousu ga Chotto Okashiinda ga. 7.195394432138937
Detective Conan OVA 04: Conan and Kid and Crystal Mother 7.195355446643682
Code Geass: Hangyaku no Lelouch - Nunnally in Wonderland 7.1952877999880105
Argento Soma 7.195181877499407
Gakusen Toshi Asterisk 7.195171305041703
Deadman Wonderland 7.194936543027922
Hal 7.194935189643855
K

Hyouka: Motsubeki Mono wa 7.177778592540552
Code:Breaker OVA 7.177767369075529
Akagami no Shirayuki-hime: Nandemonai Takaramono, Kono Page 7.177678968607864
Campione!: Matsurowanu Kamigami to Kamigoroshi no Maou 7.177669348216464
Major S1 7.177449393052686
Hanamaru Youchien 7.177381207766512
Hentai Ouji to Warawanai Neko. 7.177310601022827
Hanada Shounen-shi 7.177283264996314
Kimi no Iru Machi OVA 7.177189129515932
Tennis no Ouji-sama: Atobe kara no Okurimono - Kimi ni Sasageru Tennis Prince Matsuri 7.177122760499495
D.N.Angel 7.177071308591512
Blue Dragon 7.177047082992704
Genshiken 7.177021235180954
Keppeki Danshi! Aoyama-kun 7.176967755012128
Detective Conan OVA 12: The Miracle of Excalibur 7.176871060437999
D.C.if: Da Capo if 7.176764629193367
RideBack 7.176668439439659
Koi to Senkyo to Chocolate 7.176664554126963
Dog Days 7.1765670356225995
Harem Time The Animation 7.176549952790145
Girls und Panzer: Kore ga Hontou no Anzio-sen Desu! 7.176531372451591
Detective Conan OVA 07: A Cha

Red Ash: Gearworld 7.159136541355643
Bakugan Battle Brawlers 7.159040270152985
Gokukoku no Brynhildr 7.159027384442208
Stella no Mahou 7.158943484936791
Tsugou no Yoi Sexfriend? 7.158929476847951
Buddy Complex 7.158733920884187
Wotaku ni Koi wa Muzukashii 7.158645428222058
Ookami to Koushinryou 7.158322270002239
Itsudatte Bokura no Koi wa 10 cm Datta. 7.158237519161209
"Bungaku Shoujo" Memoire 7.158222594210911
Spiral: Suiri no Kizuna 7.1580830152876995
Kurokami The Animation 7.157827476404745
Kono Sekai no Katasumi ni 7.1576091723901145
Rakudai Kishi no Cavalry 7.157516516683899
Diamond no Ace OVA 7.1573573655069165
Aoi Umi no Tristia 7.156909620385799
Oda Nobuna no Yabou 7.15670740729576
Clannad Movie 7.156108146377099
Ange Vierge 7.155994649984796
Koutetsujou no Kabaneri 7.1559308710989775
Aki-Sora 7.155874297075613
Tokimeki Memorial: Only Love 7.155845564302117
Jewelpet Magical Change 7.1558454823368365
Gaist Crusher 7.155670293644349
To LOVE-Ru Darkness OVA 7.155458992888315
Higur

Toji no Miko 7.101343035695935
Futari wa Precure: Max Heart Movie 2 - Yukizora no Tomodachi 7.100267226478578
Air Movie 7.100022540495842
Shuffle! 7.097663574732886
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai. 7.097275456653837
Kuroinu: Kedakaki Seijo wa Hakudaku ni Somaru 7.097005919401128
K-On! Movie 7.09699989113615
Kira Kira 5th Anniversary Live Anime: Kick Start Generation 7.096251828206495
Gate Keepers 7.096003820288113
Shin Chou Kyou Ryo: Condor Hero 7.094837816707012
D.C. I&II P.S.P. Re-Animated 7.093033397642376
School Days 7.091529788176125
Jinki:Extend 7.088329013553425
Ginban Kaleidoscope 7.087607023770849
Cluster Edge 7.0868361957172
Hanasakeru Seishounen 7.086579765632908
Wild Adapter 7.086560549256493
Koukaku no Pandora: Ghost Urn 7.086536573929666
Toradora! 7.086149588868542
Darling in the FranXX 7.085948188492966
Haru no Ashioto The Movie: Ourin Dakkan 7.085155095653417
Shuumatsu Nani Shitemasu ka? Isogashii desu ka? Sukutte Moratte Ii desu ka? 7.0850114697

In [50]:
np.where(np.isin(anime_ids, [20, 5]))[0]

array([0, 7])

In [53]:
a = np.where(~np.isin(anime_ids, np.array(user_shows_in_db(cmlimmDF, animesDF))[:, 0]))[0]
a

array([   0,    1,    2, ..., 2081, 2082, 2083])

In [54]:
b = np.where(np.isin(anime_ids, np.array(user_shows_in_db(cmlimmDF, animesDF))[:, 0]))[0]
b

array([ 601,  653,  852,  855,  883,  892,  914,  917, 1029, 1036, 1053,
       1091, 1103, 1108, 1147, 1219, 1286, 1330, 1335, 1375, 1387, 1427,
       1458, 1461, 1479, 1559, 1593, 1625, 1628, 1641, 1667, 1682, 1731,
       1750, 1765, 1783, 1792, 1807, 1832, 1866, 1875, 1891, 1903, 1956,
       1962, 1968, 2004, 2014, 2017, 2021, 2028])

In [61]:
c = similarity_table[b,a[:, np.newaxis]]

In [62]:
d = similarity_table[b[:, np.newaxis],a]

In [64]:
np.array_equal(c,d)

False

In [46]:
test[np.array([0,2]),np.array([1])]

array([2, 8])